In this notebook, we will analyze game sales and console units sold to determine which gaming consoles are the most successful. Additionally, we will examine whether specific regions or game genres contribute to the success of these consoles.

Import the needed modules.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Feature:
Read in the data.

In [ ]:
console = pd.read_csv(r"data/best_selling_consoles.csv")
games = pd.read_csv(r"data/video_games_sales.csv")

Feature: Cleaning and merging the DataFrames and calculating new values.

Checking the games dataframe.

In [ ]:
games.head(5)

In [ ]:
games.tail(5)

In [ ]:
games.columns


Cleaning games column names.

In [ ]:
games.columns = games.columns.str.title() .str.strip() .str.replace('_', ' ')

Dropping unneeded games columns.

In [ ]:
games_clean = games.drop('Rank', axis=1)
games_clean.head(5)

Checking console.

In [ ]:
console.head(10)

Checking console column names.

In [ ]:
console.columns

Dropping not needed columns.

In [ ]:
console_clean = console.drop('Remarks', axis=1)

Renaming 'Console name' to 'Platform'.

In [ ]:
console_clean = console_clean.rename(columns={'Console Name': 'Platform'})
console_clean.head(10)

Checking how each dataframe lists different consoles in the platform column.

In [ ]:
console_clean['Platform'].unique()

In [ ]:
games_clean['Platform'].unique()

Matching and verifying the platforms in the console dataframe so it can merge with console dataframe. *Interesting tidbit I wrote this matching out by hand on a sheet of paper before typing to make sure that it all was exact.  

In [ ]:
platform_matching ={
    'NES': 'NES/Famicom',
    'GB': 'Game Boy',
    'DS': 'Nintendo DS',
    'X360': 'Xbox 360',
    'PS3': 'PlayStation 3',
    'PS2': 'PlayStation 2',
    'SNES': 'SNES/Super Famicom',
    'GBA': 'Game Boy Advance',
    '3DS': 'Nintendo 3DS',
    'PS4': 'PlayStation 4',
    'N64': 'Nintendo 64',
    'PS': 'PlayStation',
    'XB': 'Xbox',
    '2600': 'Atari 2600',
    'PSP': 'PlayStation Portable',
    'XOne': 'Xbox One',
    'GC':'GameCube',
    'WiiU':'Wii U',
    'GEN':'Sega Genesis/Mega Drive',
    'DC': 'DreamCast',
    'PSV': 'PlayStation 5',
    'SAT': 'Sega Saturn',
    'SCD': 'Sega CD',
    'WS':'Wonderswan',
    'TG16': 'PC Engine/TurboGrafx-16',
    'GG':'Sega Game Gear'}


games_clean['Platform'] = games_clean['Platform'].replace(platform_matching)
games_clean

Merging the dataframes on the Platform column.

In [ ]:
games_console_mergered = pd.merge(games_clean, console_clean, on='Platform', how='inner')
games_console_mergered.head(10)

Renaming columns in the merged dataframe.

In [ ]:
cols_rename = {'Type': 'Console Type',
    'Company':'Console Manufacturer',
    'Released Year': 'Console Released',
    'Discontinuation Year': 'Console Discontinued',
    'Units sold (million)': 'Consoles Sold'}

merged_col_rename = games_console_mergered.rename(columns=(cols_rename))
merged_col_rename.head(5)

Checking for nulls.

In [ ]:
merged_col_rename.isna().sum()

Filling the nulls.

In [ ]:
merged_col_rename['Year'] = merged_col_rename['Year'].fillna(0).astype(int)
merged_col_rename['Publisher'] = merged_col_rename['Publisher'].fillna('Unknown')

Creating a couple unique values from the combined dataframe.

Matching the game release year to the console release year to get the launch global sales of the games.

In [ ]:
games_yr_console_released = merged_col_rename[merged_col_rename['Year'] == merged_col_rename["Console Released"]]
games_yr_console_released.head(5)

Plot 1:
Plotting the launch year games sales.

In [ ]:
launch_year_game_sales = games_yr_console_released.groupby('Platform')['Global Sales'].sum().reset_index()

plt.figure(figsize=(12, 6))
plt.bar(launch_year_game_sales['Platform'], launch_year_game_sales['Global Sales'])
plt.title('Console Launch Year Game Sales')
plt.xlabel('Console')
plt.xticks(rotation=90)
plt.ylabel('Launch Year Game Sales(Millions)')

The Wii had a great Launch year in comparison to the other consoles.

Matching the game release year to console discontinued year to get the global sales of the games in the last year of the consoles life.

In [ ]:
games_console_discontinued = merged_col_rename[merged_col_rename['Year'] == merged_col_rename["Console Discontinued"]]
console_discontinued_games = games_console_discontinued.groupby('Platform')['Global Sales'].sum().reset_index()
console_discontinued_games

Plot 2:
Plotting the games that launched in the consoles final year.

In [ ]:
plt.figure(figsize=(12, 6))
plt.bar(console_discontinued_games['Platform'], console_discontinued_games['Global Sales'])
plt.title('Console Launch Year Game Sales')
plt.xlabel('Console')
plt.xticks(rotation=90)
plt.ylabel('Launch Year Game Sales(Millions)')

As you can see theres very few consoles that even had games that launched in last year of the consoles life. Again the Wii had the best end life just as it had the best launch year in terms of the global sales of games. 

Plot 3: 
A plot to show the number in millions of console units sold globally. 

In [ ]:
plt.figure(figsize=(12, 6))
plt.bar(merged_col_rename['Platform'], merged_col_rename["Consoles Sold"])
plt.title('Consoles Sold')
plt.xlabel('Console')
plt.xticks(rotation=90)
plt.ylabel('Units Sold (Millions)')
plt.show()

Feature: 3 plots have been made.
Don't worry though they will still be a few more plots on this journey through sales data to keep you entertained.  

Grouping global game sales by console. 

In [ ]:
global_sales_platform = merged_col_rename.groupby('Platform')['Global Sales'].sum().reset_index()
global_sales_platform


Plot 4: 
plotting global game sales by console.

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(global_sales_platform['Platform'], global_sales_platform['Global Sales'])
plt.title('Global Game Sales By Console')
plt.xlabel('Console')
plt.xticks(rotation=90)
plt.ylabel('Global Game Sales(Millions)')

The PlayStation 2 games clearly outsold all the other platforms with Xbox 360 second, WII third and the Nintendo DS all having greater success than all other Platforms game sales. 

Grouping and plotting the sales of the games by different regions. *This was one of the last plots I made for the project and discovered it was easier to plot with .plot().

In [ ]:
regional_sales = merged_col_rename.groupby('Platform')[['Na Sales', 'Eu Sales', 'Jp Sales']].sum()



regional_sales.plot(kind='bar', figsize=(12, 6), colormap= 'viridis')
plt.title('Regional Game Sales by Console', fontsize=14)
plt.xlabel('Console', fontsize= 12)
plt.ylabel('Game Sales(Millions)', fontsize= 12)
plt.tight_layout()


Looks Like regardless of the console that the North American and European regions are the biggest contributers to global sales for all platforms.  

Aggregating the console units Sold by first so the number doesn't end up being more units sold than people on the planet and the sum of the global sales to create a scatterplot. 

In [ ]:
consoles_scatter = merged_col_rename.groupby('Platform').agg({'Global Sales': 'sum', 'Consoles Sold': 'first' })



plt.figure(figsize= (12, 6))
sns.scatterplot(x='Consoles Sold', y='Global Sales', hue='Platform', data=consoles_scatter, palette='tab20', alpha= .99)
plt.title('Game Sales VS Console Sold', fontsize= 16)
plt.xlabel('Consoles Sold Worldwide (Millions)', fontsize= 14)
plt.ylabel('Global Game Sales(Millions)', fontsize=14)
plt.legend(fontsize=9, title='Platform', bbox_to_anchor=(1, 1))
plt.tight_layout()

You can see that the PlayStation 2 not only sold the most consoles but also the most games. This potentially shows that a greater console install base equates to higher sales of games. 

Feature: Create a pivot table and create a seaborn plot from the pivot.


In [ ]:
genre_pivot = pd.pivot_table(merged_col_rename, index='Genre', columns='Platform', values='Global Sales', aggfunc='sum')
genre_pivot


Filling the NaNs of the pivot with 0 to make the heatmap complete. 

In [ ]:
genre_pivot = genre_pivot.fillna(0)
genre_pivot

Using seaborn to plot the heatmap using the created pivot table.

In [ ]:
plt.figure(figsize=(18, 9))
sns.heatmap(genre_pivot, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.75)
plt.title("Global Game Sales(millions) by Genre and Console", fontsize=24)
plt.xlabel('Console', fontsize=18)
plt.ylabel('Genre', fontsize= 18)
plt.tight_layout()


See how action games and sports games contributed to the success of the Playstation 2 and that sports and misc games contributed the most to the Wiis. 